<a href="https://colab.research.google.com/github/Naveenkumar-2007/stockprice/blob/main/Untitled19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,LSTM,Dropout
import yfinance as yf

In [ ]:
stock='GOOG'
start_date="2018-01-01"
end_date="2025-08-12"
last_back=60

In [ ]:
df=yf.download(stock,start=start_date,end=end_date)

/tmp/ipython-input-823496602.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download(stock,start=start_date,end=end_date)
[*********************100%***********************]  1 of 1 completed


In [ ]:
df.head()

Price,Close,High,Low,Open,Volume
Ticker,GOOG,GOOG,GOOG,GOOG,GOOG
Date,,,,,
2018-01-02,52.935341,53.031768,51.952683,52.107263,24752000
2018-01-03,53.804180,53.993553,52.846373,52.901048,28604000
2018-01-04,53.999020,54.355403,53.879828,54.078549,20092000
2018-01-05,54.785843,54.886248,54.277364,54.376775,25582000
2018-01-08,55.019951,55.235171,54.755524,54.785842,20952000


In [ ]:
data=df[['Close']]

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
scaler_data=scaler.fit_transform(data)

In [ ]:
print(len(scaler_data))

1912


In [ ]:
x_train,y_train=[],[]
for i in range(last_back,len(scaler_data)):
  x_train.append(scaler_data[i-last_back:i,0])
  y_train.append(scaler_data[i,0])

In [ ]:
x_train,y_train=np.array(x_train),np.array(y_train)


In [ ]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

In [ ]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,activation='relu',input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam',loss='MeanSquaredError')

In [ ]:
store=model.fit(x_train,y_train,epochs=50,verbose=0,batch_size=32)

In [ ]:
import pickle
with open('scaler.pkl','wb') as f:
  pickle.dump(scaler,f)

In [ ]:
model.save('lstm.h5')